In [1]:
from transformers import AutoTokenizer, pipeline

tokenizer_gen = AutoTokenizer.from_pretrained("distilgpt2", padding_side="left")
generator = pipeline(
    "text-generation",
    "./results_causal/checkpoint-39000",
    tokenizer=tokenizer_gen,
    pad_token_id=50256,
)

tokenizer_ref = AutoTokenizer.from_pretrained("distilroberta-base", padding_side="left")
reflector = pipeline(
    "feature-extraction",
    "./results_masked/checkpoint-39000",
    tokenizer=tokenizer_ref,
    pad_token_id=50256,
)

Some weights of RobertaModel were not initialized from the model checkpoint at ./results_masked/checkpoint-39000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
samples = generator(
    text_inputs="Unsupervised",
    num_return_sequences=5000,
    max_length=25,
    truncation=True,
)

In [ ]:
import gc
del generator
del tokenizer_gen
gc.collect()

In [3]:
import numpy as np
outputs = reflector([s["generated_text"] for s in samples])
outputs = np.array([output[-1][-1] for output in outputs])

In [ ]:
del reflector
del tokenizer_ref
gc.collect()

In [4]:
from sklearn.metrics.pairwise import cosine_distances
V = np.load("vectors_samples.npy")
D = cosine_distances(outputs, V)

In [5]:
idx_novelty = D.min(axis=1) >= 0.001
idx_novelty.sum()

3542

In [6]:
idx_feasibility = (D <= 0.003).sum(axis=1) >= 50
idx_feasibility.sum()

4872

In [7]:
idx_objective = idx_novelty & idx_feasibility
idx_objective.sum()

3414

### Results

In [8]:
import pandas as pd
pd.set_option("max_colwidth", 200)

results = pd.DataFrame(samples)
results["novelty_score"] = D.min(axis=1) / D.min(axis=1).std()
results["feasibility_score"] = (D <= 0.003).sum(axis=1) / (D <= 0.003).sum(axis=1).std()
results["reflection_score"] = np.sqrt((D <= 0.003).sum(axis=1) * D.min(axis=1))
results["passed_threshold"] = idx_objective
results = (
    results.sort_values("reflection_score", ascending=False)
    .reset_index(drop=True)
    .round(3)
)

#### Best Generated Text

*Reasonably novel and very feasible*

In [9]:
results[results["passed_threshold"]].head(10)

,generated_text,novelty_score,feasibility_score,reflection_score,passed_threshold
0,"Unsupervised Generative Transformer Network for Time Series Generation: Improved Accuracy, Constraint Bias and Interpretable Guarant",0.335,3.816,3.783,True
1,Unsupervised Domain Adaptation using Transfer Learning with Variational Autoencoder Embeddings with Application to Edge Applications and,0.362,3.486,3.757,True
2,"Unsupervised Subspace Clustering for Speech Recognition Using Generative Models: A Case Study in the UK, Italy",0.396,3.123,3.723,True
3,Unsupervised Learning of Dynamic Local Kernels Algorithms for Visual Analytics: A New Perspective From the Perspective of Sparse,0.424,2.914,3.719,True
4,Unsupervised Learning from Unlabeled Data Using Bayesian Neural Networks: How to Achieve the Best of Both Worlds? A,0.489,2.515,3.711,True
5,Unsupervised Deep Reinforcement Learning for Unsupervised Feature Extraction in Automated Time Series Prediction Using Neural Networks: A,0.390,3.139,3.705,True
6,Unsupervised Deep Learning with Gradient-invariant Feature Selection for Imbalanced Data Systems: Application to Machine Learning and,0.355,3.435,3.694,True
7,Unsupervised Prediction of COVID-19 with Transductive Feature Embeddings as a Neural Network Model with Sparse,0.348,3.496,3.693,True
8,Unsupervised Text Classification With Graph Convolutional Network: A Case Study in the Use of Unsupervised Objectives Towards,0.377,3.229,3.693,True
9,Unsupervised and Hierarchical Learning and Uncertainty Detection for Natural Language Processing Time Series Data through a Framework of Deep,0.378,3.215,3.689,True


#### Worst Generated Text

*Novel but infeasible*

In [10]:
results[~results["passed_threshold"]].tail(10)[::-1]

,generated_text,novelty_score,feasibility_score,reflection_score,passed_threshold
4999,Unsupervised Graph-based Convolutional Transformers for Human Action Recognition in Mobile Devices using Deep Network Training Models (DS,1.449,0.0,0.0,False
4998,Unsupervised Deep Learning on Time Series Data for Predictive Control of the Vehicle Driving Simulator (VTA) Prediction System (,1.182,0.0,0.0,False
4997,Unsupervised Neural Translation Transformers for Non-Supervised Speech Recognition with Latent Variable Transformation (TRT2-D,1.580,0.0,0.0,False
4996,Unsupervised Pre-training with Attention-based Memory Models for Anomaly Detection in Internet of Things (IoT II,0.898,0.0,0.0,False
4995,Unsupervised Transfer Learning in Continuous Time for Visual Scene Analysis of Pedestrians Through Dynamic Contextual Networks (DAR-,1.317,0.0,0.0,False
4994,Unsupervised Inference with Gradient Descent via Deep Convolutional Neural Networks for Speech Recognition (VDAD,1.625,0.0,0.0,False
4993,Unsupervised learning based on sparse weight classification: An evaluation study for the presence of missing data in the dataset and beyond (,1.133,0.0,0.0,False
4992,Unsupervised Deep Learning on Graphs Using Generative Adversarial Networks: A Comparison of Methods and Experiments Beyond.,16.915,0.0,0.0,False
4991,Unsupervised Online Learning in Neural Networks for Classification of Spine Genomic Data Using an Ensemble of Machine Learning Models (,1.076,0.0,0.0,False
4990,Unsupervised Learning of Low Rank Objects from Human Eye-View Images Using a Deep Q-Learning Framework (DQF,1.541,0.0,0.0,False
